In [1]:
import bs4
import json
import requests
from bs4 import BeautifulSoup

link = "https://secure.meetcontrol.com/divemeets/system/index.php"


In [2]:
html = requests.get(link).text
soup = BeautifulSoup(html, "html.parser")
tabs = (
    soup.find("div", {"id": "dm_menu_centered"})
    .find("ul")
    .find_all("li", recursive=False)
)
tab_dict = {tab.find("a", recursive=False).text: tab for tab in tabs}


In [3]:
seen_orgs = dict()

meets = (
    tab_dict["Meets"]
    .find("ul", recursive=False)
    .find("li", recursive=False)
    .find_all("a")[1:]
)
upcoming_meets = {m.text.strip(): m["href"] for m in meets}


In [4]:
pmeets = (
    tab_dict["Meets"]
    .find("ul", recursive=False)
    .find_all("li", recursive=False)[1]
    .find_all("a")[1:]
)
past_meets = dict()
last_key = None
for a in pmeets:
    if a["href"] == "#":
        last_key = a.text
        past_meets[last_key] = dict()
    else:
        past_meets[last_key][a.text.strip()] = a["href"]


In [5]:
# with open("upcoming_meets.json", "w") as f:
#     json.dump(upcoming_meets, f)

# with open("past_meets.json", "w") as f:
#     json.dump(past_meets, f)


In [6]:
parse_link ="https://secure.meetcontrol.com/divemeets/system/profile.php?number=56961" 
soup = BeautifulSoup(requests.get(parse_link).text, "html.parser")
result = soup.find("td").text
name_split = result.split("Name: ")[1].split("City/State: ")
name = name_split[0]
city_split = name_split[1].split("Country: ")
city = city_split[0]
country_split = city_split[1].split("Gender: ")
country = country_split[0]
age_split = country_split[1].split("FINA Age: ")
gender_split = age_split[0].split("Age: ")
gender = gender_split[0]
age = gender_split[1]
fina_age_split = age_split[1].split("High School Graduation: ")
fina_age = fina_age_split[0]
hs_grad_split = fina_age_split[1].split("DiveMeets #: ")
hs_grad = hs_grad_split[0]
dm_num = hs_grad_split[1].split("Diving:")[0]

In [7]:
items = []
for i in soup.td.contents:
    if i == '\n':
        continue
    if isinstance(i, bs4.element.NavigableString):
        items.append(i.strip())
    elif i.name == 'strong':
        items.append(i.text.strip())
    elif i.name == 'a':
        items.append((i.text.strip(), i['href']))

items

# ignores "Diver Profile" header for now
i = 1
# Looks for "Diving:" line to start parsing teams instead of personal info
found_diving = False
new_items = {}
orgs = {}
while i < len(items):
    if items[i] == "Diving:":
        i += 1
        found_diving = True
    elif not found_diving:
        new_items[items[i][:-1]] = items[i+1]
        i += 2
    else:
        coach, link = items[i+3]
        link = 'https://secure.meetcontrol.com/divemeets/system/' + link
        org = {"Team":items[i+1], "Coach":coach, "Link": link}
        orgs[items[i][:-1]] = org
        i += 4
new_items["Organizations"] = orgs

new_items


{'Name': 'Logan Sherwin',
 'City/State': 'Harrison City PA',
 'Country': 'United States',
 'Gender': 'M',
 'Age': '21',
 'FINA Age': '22',
 'High School Graduation': '2019',
 'DiveMeets #': '56961',
 'Organizations': {'Amateur Athletic Union (AAU)': {'Team': 'Pitt Aquatic Club (Diver)',
   'Coach': 'Santos Jesus',
   'Link': 'https://secure.meetcontrol.com/divemeets/system/profilec.php?number=56797'},
  'National Collegiate Athletic Association (NCAA)': {'Team': 'University of Chicago (Diver)',
   'Coach': 'Benson Becky',
   'Link': 'https://secure.meetcontrol.com/divemeets/system/profilec.php?number=16241'},
  'USA Diving': {'Team': 'Unattached (Diver)',
   'Coach': 'Santos Jesus',
   'Link': 'https://secure.meetcontrol.com/divemeets/system/profilec.php?number=56797'}}}